In [ ]:
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import json
import os
from tqdm import tqdm
import time

# Variables pour la taille des images
x, y = 20, 30

folder_name = f"images_{x}x{y}"
os.makedirs(folder_name, exist_ok=True)

print("📂 Chargement du dataset...")
with open("movies_full_classif.json") as dataset:
    data = json.load(dataset)

print(f"🎬 {len(data)} films trouvés dans le dataset")

movies_to_process = data[:5]
print(f"🔄 Traitement de {len(movies_to_process)} images...")

imgs = []
genres = []
failed_downloads = []

with tqdm(movies_to_process, desc="Processing images", unit="img") as pbar:
    
    for i, movie in enumerate(pbar):
        try:
            pbar.set_description(f"Genre: {movie['genre'][:15]}")
            
            url = movie["poster"]
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            img = Image.open(BytesIO(response.content)).convert("RGB")
            img = img.resize((x, y))
            
            genre_clean = movie['genre'].replace('/', '_').replace('\\', '_')
            filename = f"{folder_name}/movie_{i:03d}_{genre_clean}.jpg"
            img.save(filename, "JPEG")
            
            img_array = np.array(img, dtype=np.float32)
            img_array /= 255.0
            img_array = np.transpose(img_array, (2, 0, 1))
            img_batch = np.expand_dims(img_array, axis=0)
            
            imgs_as_lists = [img[0].flatten().tolist() for img in imgs]
            
            imgs.append(img_batch)
            genres.append(movie["genre"])
            
            pbar.set_postfix({
                'Success': len(imgs),
                'Failed': len(failed_downloads),
                'Size': f"{x}x{y}"
            })
            
        except requests.RequestException as e:
            failed_downloads.append({
                'index': i,
                'genre': movie.get('genre', 'Unknown'),
                'error': f"Network error: {str(e)}"
            })
            pbar.set_postfix({
                'Success': len(imgs),
                'Failed': len(failed_downloads),
                'LastError': 'Network'
            })
            
        except Exception as e:
            failed_downloads.append({
                'index': i,
                'genre': movie.get('genre', 'Unknown'),
                'error': f"Processing error: {str(e)}"
            })
            pbar.set_postfix({
                'Success': len(imgs),
                'Failed': len(failed_downloads),
                'LastError': 'Processing'
            })
        
        time.sleep(0.1)

# Résumé final
print(f"\n✅ Traitement terminé !")
print(f"   📊 Images traitées avec succès: {len(imgs)}")
print(f"   ❌ Échecs: {len(failed_downloads)}")
print(f"   📐 Taille des images: {x}x{y}")
print(f"   🎯 Genres uniques: {len(set(genres))}")

# Afficher les erreurs si il y en a
if failed_downloads:
    print(f"\n⚠️  Détail des échecs:")
    for fail in failed_downloads[:5]:  # Afficher les 5 premiers
        print(f"   - Film {fail['index']} ({fail['genre']}): {fail['error']}")
    if len(failed_downloads) > 5:
        print(f"   ... et {len(failed_downloads) - 5} autres échecs")

print(f"\n💾 Arrays prêts pour l'entraînement:")
print(f"   - imgs: {len(imgs)} images de shape {imgs[0].shape if imgs else 'N/A'}")
print(f"   - genres: {len(genres)} labels")

📂 Chargement du dataset...
🎬 16849 films trouvés dans le dataset
🔄 Traitement de 5 images...


Genre: Action: 100%|██████████| 5/5 [00:02<00:00,  2.32img/s, Success=5, Failed=0, Size=20x30]


✅ Traitement terminé !
   📊 Images traitées avec succès: 5
   ❌ Échecs: 0
   📐 Taille des images: 20x30
   🎯 Genres uniques: 1

💾 Arrays prêts pour l'entraînement:
   - imgs: 5 images de shape (1, 3, 30, 20)
   - genres: 5 labels


In [4]:
print(failed_downloads)

[{'index': 1986, 'genre': 'Aventure', 'error': "Network error: HTTPSConnectionPool(host='image.tmdb.org', port=443): Read timed out. (read timeout=10)"}, {'index': 5158, 'genre': 'Documentaire', 'error': "Network error: HTTPSConnectionPool(host='image.tmdb.org', port=443): Read timed out. (read timeout=10)"}, {'index': 5452, 'genre': 'Documentaire', 'error': "Network error: HTTPSConnectionPool(host='image.tmdb.org', port=443): Read timed out. (read timeout=10)"}, {'index': 6119, 'genre': 'Drame', 'error': "Network error: HTTPSConnectionPool(host='image.tmdb.org', port=443): Read timed out. (read timeout=10)"}, {'index': 9403, 'genre': 'Histoire', 'error': "Network error: HTTPSConnectionPool(host='image.tmdb.org', port=443): Read timed out. (read timeout=10)"}, {'index': 9481, 'genre': 'Histoire', 'error': "Network error: HTTPSConnectionPool(host='image.tmdb.org', port=443): Read timed out. (read timeout=10)"}, {'index': 9580, 'genre': 'Histoire', 'error': "Network error: HTTPSConnectio

In [5]:
# Conversion des images en listes pour JSON
# imgs_as_lists = [img.tolist() for img in imgs]

# # Création d'une structure avec les genres associés
# data_to_save = [
#     {"image": img, "genre": genre}
#     for img, genre in zip(imgs_as_lists, genres)
# ]

# # Écriture dans un fichier JSON
# with open("images_and_genres.json", "w") as f:
#     json.dump(data_to_save, f)

imgs_as_lists = [img[0].flatten().tolist() for img in imgs]
data_to_save = [
    {"image": img, "genre": genre}
    for img, genre in zip(imgs_as_lists, genres)
]

# Sauvegarder dans un fichier JSON
with open("images_flat_full.json", "w") as f:
    json.dump(data_to_save, f)
